In [3]:
%pylab
%matplotlib inline

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
#Reload changes -> always run this
%load_ext autoreload
%autoreload 2

## Homework 10 

Ron Cordell, Ted Dunmire, Filip Krunic 

-----

##### Setup 

To have IPython correctly use PySpark, we import it as a module using the `findspark` library. 

In [4]:
import findspark
import os
findspark.init()

import pyspark
sc = pyspark.SparkContext()

-----

### Question 10.1

*In Spark write the code to count how often each word appears in a text document (or set of documents). Please use this homework document as a the example document to run an experiment.  Report the following: provide a sorted list of tokens in decreasing order of frequency of occurence.*

#### Solution:

We first load the text file from a local directory. Once it's placed in a Spark context, it goes through two mapping and one reduce phase which collects all of the terms and returns their count. This is accomplished with `flatMap` which generates key-value pairs using `.split()` to get the tokens. The second map phase assigns `1` to the value for each key. Finally, the data is reduced by key and returned with `.collect()`. 

In [5]:
# Load
data = sc.textFile('homework_10.txt')

# Split and tokenize 
tokens = data.flatMap(lambda x: x.split()).map(lambda x: (x, 1))

# Count 
wordCounts = tokens.reduceByKey(lambda a, b: a + b)

# Return 
wordCounts.collect()

[(u'y)', 1),
 (u'sameModel', 1),
 (u'all', 1),
 (u'weight(X)=', 1),
 (u'10.2:', 1),
 (u'What', 4),
 (u'not', 1),
 (u'intuitoive', 1),
 (u'(cluster', 1),
 (u'consists', 1),
 (u'-----------------------', 2),
 (u'based', 1),
 (u'parameters', 1),
 (u'error(point):', 1),
 (u'(Euclidean', 1),
 (u'Modify', 1),
 (u'3', 1),
 (u'Here', 1),
 (u'languages', 1),
 (u'snippet', 1),
 (u'SQRT(X.X)=', 1),
 (u'2,', 1),
 (u'weeks', 1),
 (u'submissions', 1),
 (u'homework', 2),
 (u'case', 1),
 (u'return', 1),
 (u'runs=10,', 1),
 (u'KMeans,', 1),
 (u'load', 1),
 (u'notebook:', 2),
 (u'experiment.', 1),
 (u'words', 3),
 (u'homeworks', 1),
 (u'compute', 1),
 (u'sc.textFile("kmeans_data.txt")', 1),
 (u'bringing', 1),
 (u'Spark', 3),
 (u'(one', 2),
 (u'runs', 1),
 (u'resource', 1),
 (u'questions===', 1),
 (u'array([float(x)', 1),
 (u'evaluation', 2),
 (u'frequency.', 1),
 (u'(point', 1),
 (u'1/||X||,', 1),
 (u'--', 2),
 (u'(or', 1),
 (u'SPECIAL', 1),
 (u'Learning', 1),
 (u'KMeansModel.load(sc,', 1),
 (u'where', 

-----

### Question 10.1.1

*Modify the above word count code to count words that begin with lower case letters (a-z) and report your findings. Again sort the output words in decreasing order of frequency.*

#### Solution: 

We use similar code as above, but include an additional map phase to remove tokens that do not begin with lower-case letters. 

In [6]:
import string 
letters = string.ascii_lowercase

# Filter 
lower = tokens.filter(lambda x: x[0][0] in letters)

# Count 
wordCounts = lower.reduceByKey(lambda a, b: a + b)

# Return 
wordCounts.collect()

[(u'y)', 1),
 (u'sameModel', 1),
 (u'all', 1),
 (u'homeworks', 1),
 (u'intuitoive', 1),
 (u'consists', 1),
 (u'snippet', 1),
 (u'based', 1),
 (u'parameters', 1),
 (u'error(point):', 1),
 (u'had', 1),
 (u'languages', 1),
 (u'weeks', 1),
 (u'submissions', 1),
 (u'homework', 2),
 (u'count', 3),
 (u'return', 1),
 (u'runs=10,', 1),
 (u'load', 1),
 (u'runs', 1),
 (u'experiment.', 1),
 (u'words', 3),
 (u'report', 3),
 (u'compute', 1),
 (u'sc.textFile("kmeans_data.txt")', 1),
 (u'bringing', 1),
 (u'point:', 1),
 (u'notebook:', 2),
 (u'resource', 1),
 (u'questions===', 1),
 (u'array([float(x)', 1),
 (u'evaluation', 2),
 (u'frequency.', 1),
 (u'list', 2),
 (u'where', 1),
 (u'initializationMode="random")', 1),
 (u'generation', 1),
 (u'provided).', 1),
 (u'run', 2),
 (u'regression.', 2),
 (u'sqrt(sum([x**2', 1),
 (u'for', 11),
 (u'please', 1),
 (u'per', 2),
 (u'cell', 1),
 (u'blanks:', 1),
 (u'above', 1),
 (u'at:', 1),
 (u'math', 1),
 (u'available', 3),
 (u'lazy', 3),
 (u'https://www.dropbox.com/s

-----

### Question 10.2

*Run the following code snippet and list the clusters that your find and compute the Within Set Sum of Squared Errors for the found clusters. Comment on your findings.*

In [7]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

# Load and parse the data
# NOTE  kmeans_data.txt is available here 
#          https://www.dropbox.com/s/q85t0ytb9apggnh/kmeans_data.txt?dl=0 
data = sc.textFile("kmeans_data.txt")  
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10,
        runs=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

# Save and load model
clusters.save(sc, "myModelPath")
sameModel = KMeansModel.load(sc, "myModelPath")

Within Set Sum of Squared Error = 0.692820323028


### Question 10.3

*Download the following KMeans notebook:*

https://www.dropbox.com/s/3nsthvp8g2rrrdh/EM-Kmeans.ipynb?dl=0

*Generate 3 clusters with 100 (one hundred) data points per cluster (using the code provided). Plot the data.
Then run MLlib's Kmean implementation on this data  and report your results as follows:*

  -- *plot the resulting clusters after 1 iteration, 10 iterations, after 20 iterations, after 100 iterations.*
  -- *in each plot please report the Within Set Sum of Squared Errors for the found clusters. Comment on the progress of this measure as the KMEans algorithms runs for more iterations*
  
#### Solution 

First, we must generate our points to be clustered. Below is the code used to accomplish this. 

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle 


# Generate points 
x = np.random.normal(loc=-5, scale=1, size=(100, 2))
y = np.random.normal(loc=0, scale=1, size=(100, 2))
z = np.random.normal(loc=5, scale=1, size=(100, 2))


# Write to file 
with open('comparison.txt', 'w') as f: 

    for point in x: 
        f.write('%s,%s\n' % (point[0], point[1]))
        
    for point in y: 
        f.write('%s,%s\n' % (point[0], point[1]))
        
    for point in z: 
        f.write('%s,%s\n' % (point[0], point[1]))        

Once we have generated our points, we need to compute KMeans over the data. We will use our custom implementation to accomplish this. This includes classes to assign the clusters, update the clusters based on new assignment, and compute errors. Finally, we have a driver that wraps this processes together. 

##### Assigning Clusters 

Here, we assign each point a cluster based on which one it is closest to. 

In [30]:
%%writefile assign_clusters.py
from __future__ import division

from mrjob.job import MRJob
from mrjob.step import MRStep

import cPickle as pickle
import numpy as np

from collections import defaultdict

class assignClusters(MRJob):

	def __init__(self, args):
		MRJob.__init__(self, args)

	def configure_options(self):
		super(assignClusters, self).configure_options()
		self.add_file_option('--centroids', 
			help='pointer to centroids file. See main runner for details.')

	def load_options(self, args):

		""" This function initializes arguments defined in 'configure_options'. """

		super(assignClusters, self).load_options(args)
		
		# Load scores 
		with open(self.options.centroids, 'r') as f: 
			self.clusters = pickle.load(f)


	def mapper_diff_comp(self, _, line):

		body = [float(x) for x in line.split(',')]
		custID = hash(''.join([str(x) for x in body]))

		# Compute distances 
		for clusterID, cluster in self.clusters.iteritems(): 
			
			cluster = np.array(cluster)
			body = np.array(body)
			weight = np.linalg.norm(body)
			dist = np.linalg.norm(body-cluster) / weight

			yield custID, [clusterID, dist]


	def reducer_find_min_cluster(self, custID, distArray):

		# Find closest cluster
		distArray = np.array(list(distArray))
		clusterIndex = np.argmin(distArray[:, 1])
		clusterID = distArray[clusterIndex, 0]

		yield custID, clusterID

	def steps(self):
		return [MRStep(mapper=self.mapper_diff_comp, 
						reducer=self.reducer_find_min_cluster)]

if __name__ == '__main__':
	assignClusters.run()

Writing assign_clusters.py


##### Update Clusters 

Once the clusters are assigned, they centers need to be updated so a new phase of iteration can begin. 

In [32]:
%%writefile update_centroids.py
from __future__ import division

from mrjob.job import MRJob
from mrjob.step import MRStep

import cPickle as pickle
import numpy as np
from collections import defaultdict

class updateCentroids(MRJob):

	def __init__(self, args):
		MRJob.__init__(self, args)

	def configure_options(self):
		super(updateCentroids, self).configure_options()

		self.add_file_option('--scores', 
			help='pointer to scores file. See main runner for details.')

		self.add_file_option('--centroids', 
			help='pointer to centroids file. See main runner for details.')

		self.add_passthrough_option('--errors', 
			default=False, help='If True, computes RMSE instead of updating the centroid centers.')


	def load_options(self, args):

		""" This function initializes arguments defined in 'configure_options'. """

		super(updateCentroids, self).load_options(args)

		self.errors = self.options.errors

		
		# Load scores 
		with open(self.options.scores, 'r') as f: 
			self.scores = pickle.load(f)


		# Load centroids
		with open(self.options.centroids, 'r') as f: 
			self.centroids = pickle.load(f)


	def mapper_telegraph_id(self, _, line):

		body = [float(x) for x in line.split(',')]
		custID = hash(''.join([str(x) for x in body]))

		# Get individual
		label = self.scores[custID]
		yield label, body


	def reducer_emit_clusters(self, label, arrays):
		
		# Compute new cluster 
		arrays = list(arrays)
		newCluster = [np.mean(x) for x in zip(*arrays)]

		yield label, newCluster


	def reducer_compute_dist(self, label, vectors):
		vectors = np.array(list(vectors))

		cluster = self.centroids[label]
		dist = sum([np.linalg.norm(body-cluster) for body in vectors])

		yield None, dist


	def reducer_rmse(self, _, dists):

		# Compute
		N = len(self.scores)
		RMSE = np.sqrt(sum(dists) / N)

		yield None, RMSE			


	def steps(self):

		""" Checks for the error flag to compute RMSE."""

		if self.errors: 

			return [MRStep(mapper=self.mapper_telegraph_id, 
							reducer=self.reducer_compute_dist), 

					MRStep(reducer=self.reducer_rmse)]

		else:

			return [MRStep(mapper=self.mapper_telegraph_id,
							reducer=self.reducer_emit_clusters)]

if __name__ == '__main__':
	updateCentroids.run()

Overwriting update_centroids.py


##### Driver 

The driver file will manage the iteration process and prints the points as we iterations progress. 

In [1]:
%%writefile kmeans_driver.py
from __future__ import division

from mrjob.job import MRJob
from mrjob.step import MRStep

from assign_clusters import assignClusters
from update_centroids import updateCentroids

import cPickle as pickle
from collections import defaultdict

import numpy as np 
import matplotlib.pyplot as plt


# Storage files 
trackerFile = './.tracker'
centroidFile = './.clusters'
scoreFile = './.scores'
dataFile = 'comparison.txt'


def getName(obj, namespace):
	return [name for name in namespace if namespace[name] is obj]


def extractValues(job, runner):
	output = defaultdict(int)
	for line in runner.stream_output(): 
		key, value = job.parse_output_line(line)
		output[key] = value

	return output 


def dumpToFile(variable, filename):
	with open(filename, 'w') as f: 
		pickle.dump(variable, f)


def dumpToTracker(variable, filename):
	with open(filename, 'a') as f: 
		f.write('dumping...' + '\n')
		f.write(str(variable) + '\n')
		f.write('dump complete.' + '\n')


def runJob(method, args, dFile=centroidFile):
	job = method(args=args)

	methodName = getName(method, globals())[0]
	print '\n\t' + 'Running ' + methodName + '...'

	with job.make_runner() as runner: 

		# Surpress console 
		runner.run()

		result = extractValues(job, runner)
		dumpToTracker(result, trackerFile)

		print '\t' + 'Complete: ' + methodName

		if dFile: 			
			dumpToFile(result, dFile)	

		else:
			return result


if __name__ == '__main__':

	# Clear files 
	open(trackerFile, 'w').close()
	open(centroidFile, 'w').close()
	open(scoreFile, 'w').close()

	# Step 1: Create initial clusters 
	with open(centroidFile, 'w') as f: 
		
		centroidDict = {
			
			'A': [0, 0]
			, 'B' : [6, 3]
			, 'C' : [3, 6]

		}

		pickle.dump(centroidDict, f)
		

	maxIter = 10

	# Loop
	for i in range(maxIter):

		print '\n' + 'Iteration ' + str(i) + '.'

		# Score based on clusters 
		centroidArg = '--centroids='+centroidFile
		runJob(assignClusters, args=[dataFile, centroidArg], dFile=scoreFile)

		# Update clusters 
		scoreArg = '--scores='+scoreFile
		runJob(updateCentroids, args=[dataFile, centroidArg, scoreArg])

		# Report errors 
		error = runJob(updateCentroids, args=[dataFile, centroidArg, scoreArg, '--errors=True'], dFile=None)
		error = error.values()[0]

		print '\nRMSE: %s' % (round(error, 5))

# 		# Load 
# 		data = []
# 		with open(dataFile, 'r') as f: 
# 			for line in f.readlines(): 

# 				line = line.strip('\n').split(',')
# 				data.append(line)

# 		# Convert 
# 		data = np.array(data)
		
# 		with open(centroidFile, 'r') as f: 
# 			clusters = pickle.load(f)

# 		clusters = np.array(clusters.values()) 


# 		# Plot 
# 		plt.plot(data[:, 0], data[:, 1], 'bo', clusters[:, 0], clusters[:, 1], 'ro')
# 		plt.show()
		

	# Print clusters
	with open(centroidFile, 'r') as f: 
		centers = pickle.load(f)

		for k, v in centers.iteritems():
			print '\n' + 'Cluster: %s' % k
			print '\t' + str(v)

Overwriting kmeans_driver.py


##### Execute 

We run the code for KMeans. 

In [2]:
!python kmeans_driver.py


Iteration 0.

	Running assignClusters...
No handlers could be found for logger "mrjob.runner"
	Complete: assignClusters

	Running updateCentroids...
	Complete: updateCentroids

	Running updateCentroids...
	Complete: updateCentroids

RMSE: 1.68977

Iteration 1.

	Running assignClusters...
	Complete: assignClusters

	Running updateCentroids...
	Complete: updateCentroids

	Running updateCentroids...
	Complete: updateCentroids

RMSE: 1.7355

Iteration 2.

	Running assignClusters...
	Complete: assignClusters

	Running updateCentroids...
	Complete: updateCentroids

	Running updateCentroids...
	Complete: updateCentroids

RMSE: 1.72347

Iteration 3.

	Running assignClusters...
	Complete: assignClusters

	Running updateCentroids...
	Complete: updateCentroids

	Running updateCentroids...
	Complete: updateCentroids

RMSE: 1.68313

Iteration 4.

	Running assignClusters...
	Complete: assignClusters

	Running updateCentroids...
	Complete: updateCentroids

	Running updateCentroids...
	Complete: upda

The progression of the RMSE is as one would expect. After about 8 Iterations, the RMSE remains unchanged, which suggests a local minimum has been reached. At this point, the cluster centers will remain unchanged for the remaining iterations. 

-----

### Question 10.4 

*Using the KMeans code (homegrown code) provided repeat the experiments in HW10.3. Comment on any differences between the results in HW10.3 and HW10.4. Explain.*

#### Solution

Using the Spark implementation of KMeans, we run the algorithm easily: 

In [7]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

# Load and parse the data
# NOTE  kmeans_data.txt is available here 
#          https://www.dropbox.com/s/q85t0ytb9apggnh/kmeans_data.txt?dl=0 
data = sc.textFile("comparison.txt")  
parsedData = data.map(lambda line: array([float(x) for x in line.strip('\n').split(',')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 3, maxIterations=10,
        runs=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE / 300))

Within Set Sum of Squared Error = 1.25632826978


There are two striking differences between the Spark methodology and the MRJob implemetation. The first is that Spark finishes faster, even without using Hadoop. The second is that the MRJob implementation has slightly lower RMSE, but I suspect that is due to the convergence criteria being met. In the MRJob implementation, there is no stopping criteria. We see that an RMSE of `1.25` is reached somewhere between Iteration 4 and 5 for MRJob. 